In [19]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
import re

In [23]:
url='https://www.jobbank.gc.ca/jobsearch/jobsearch?flg=E&fcan=1&fss=1&fss=C&fwcl=B&fwcl=C&fwcl=D&fwcl=E&fglo=1&sort=M&fwht=D&fwht=M&fsrc=16&fjnc=1&fexp=0&fexp=1&fprov=AB&fprov=ON&fskl=101020&fskl=101010'
page=requests.get(url)

soup=BeautifulSoup(page.content, 'html.parser')

job_elements=soup.find_all('a', class_='resultJobItem')

In [41]:
role_list=[]
dates_list=[]
business_list=[]
locations_list=[]
salaries_list=[]
id_list=[]

jobs={
    
    'ID':id_list,
    'ROLES':role_list,
    'SALARY':salaries_list,
    'EDUCATION':education_list,
    'EXPERIENCE':experience_list,
    'NOC CODE':noccode_list,
    'BUSINESS':business_list,
    'LOCATIONS':locations_list,
    'PUBLISHED DATES':dates_list,
    'EMPLOYMENT TYPE':employment_type_list,
    'JOB DURATION':job_duration_list,
    'ADRESS':adress_list,
    'WEEKLY HOURS':weekly_hours_list,
    'VACANCIES':vacancies_list,
    
}

for job_list in job_elements:
    
    role=job_list.find('span', class_='noctitle')
    fixed_role=role.find(string=True, recursive=False).strip() #Cuando usas Recursive, no pone los datos de los demás "span".
    role_list.append(fixed_role)
    
    published_date=job_list.find('li', class_='date')
    fixed_date=published_date.find(string=True, recursive=False).strip() 
    dates_list.append(fixed_date)

    business_name=job_list.find(class_='business')
    fixed_business=business_name.find(string=True, recursive=False).strip()
    business_list.append(fixed_business)

    location=job_list.find('li', class_='location').get_text()
    fixed_location=location[10:].strip()
    locations_list.append(fixed_location)

    salary=job_list.find(class_='salary').get_text()
    fixed_salary=salary[30:].strip()
    if 'hourly' in fixed_salary:
        fix_sal=fixed_salary[:5]
    if 'anually' in fixed_salary:
        fix_sal=(fixed_salary[:9].replace(",",""))/2000
    salaries_list.append(f"$"+fix_sal)

for link in soup.find_all('a'): #CHANGE THIS FOR SOMETHING BETTER
    id=link.get('href')
    try:
        id.split('/')[-1:]
        id_number=id[22:30]
        try:
            id_number=int(id_number)
            id_list.append(id_number)
        except:
            pass
    except:
        pass
        
        

#print(str(dates_list)+'\n')
#print(str(role_list)+'\n')
#print(str(business_list)+'\n')
#print(str(locations_list)+'\n')
#print(str(salaries_list)+'\n')
#print(id_list)

In [45]:
noccode_list=[]
employment_type_list=[]
job_duration_list=[]
education_list=[]
experience_list=[]
adress_list=[]
weekly_hours_list=[]
vacancies_list=[]


#for ids in id_list:
try:
    id_url='https://www.jobbank.gc.ca/jobsearch/jobposting/41460166'#+str(ids)
    id_page=requests.get(id_url)
    id_soup=BeautifulSoup(id_page.content, 'html.parser')
    job_details=id_soup.find('main', class_='container')

    #NOC CODE
    noccode=job_details.find('span', class_='aa_jobbank_job_noccode').get_text()
    noccode_list.append(noccode)

    #SHIFT TYPE
    employment_type=job_details.find('span', property='employmentType')
    fixed_employment_type=employment_type.find(string=True, Recursive=False).get_text()
    employment_type_list.append(fixed_employment_type)

    #JOB DURATION
    for duration in employmentType:
        job_duration=duration.get_text(strip=True) 
        job_duration_list.append(job_duration)
        break

    #WEEKLY HOURS
    weekly_hours=job_details.find('span', property='workHours').get_text()
    weekly_hours_list.append(weekly_hours)

    #FULL ADRESS
    adress=job_details.find('span', property='streetAddress').get_text()
    adress_list.append(adress)

    #VACANCIES
    vacancies=job_details.find('span', class_='fa fa-user').get_text()
    vacancies_list.append(vacancies)
    
    description=job_details.find('span', property='description').get_text()

    #EDUCATION
    education=re.search(r'Education: ([^.]+)', description)
    fixed_education=education.group(1)
    education_list.append(fixed_education)

    #EXPERIENCE
    experience=re.search(r'Experience: ([^.]+)', description)
    fixed_experience=experience.group(1)
    experience_list.append(fixed_experience)

except:
    pass

#print(noccode_list)
#print(employment_type_list)
#print(job_duration_list)
#print(education_list)
#print(experience_list)
print(weekly_hours_list)
print(adress_list)
print(vacancies_list)

[]
[]
[]


In [35]:
#print(jobs)